# train wordvec

In [1]:
from ML_utils import ML_utils
import shutil
import os
import numpy as np
from gensim.models import word2vec
import pandas as pd
import re

In [ ]:
userdict_path = './jieba_dict/ptt_userdict.txt'
jieba_dict = 'jieba_dict/jieba_merge.txt'
dir_path = r"./word2vec_model"
ML_utils.CallF_DirFile_save(dir_path, ML_utils.Jieba_file_segmentation, 
                            replace_old=False,regular=True,file_filter_=None,
                            dict_path=jieba_dict,load_userdict_path=userdict_path,HMM=False)

Building prefix dict from D:\Backup\ml_data\GitHub\chat-robot_seq2seq\jieba_dict\jieba_merge.txt ...
2019-05-11 00:34:49,698 : DEBUG : Building prefix dict from D:\Backup\ml_data\GitHub\chat-robot_seq2seq\jieba_dict\jieba_merge.txt ...
Dumping model to file cache C:\Users\Jhin\AppData\Local\Temp\jieba.uce9186881637619ca72120272e27f9ca.cache
2019-05-11 00:34:51,152 : DEBUG : Dumping model to file cache C:\Users\Jhin\AppData\Local\Temp\jieba.uce9186881637619ca72120272e27f9ca.cache
Loading model cost 1.633 seconds.
2019-05-11 00:34:51,333 : DEBUG : Loading model cost 1.633 seconds.
Prefix dict has been built succesfully.
2019-05-11 00:34:51,334 : DEBUG : Prefix dict has been built succesfully.
2019-05-11 00:34:51,675 : INFO : 已完成前 10000 行的斷詞
2019-05-11 00:34:52,016 : INFO : 已完成前 20000 行的斷詞
2019-05-11 00:34:52,365 : INFO : 已完成前 30000 行的斷詞
2019-05-11 00:34:52,720 : INFO : 已完成前 40000 行的斷詞
2019-05-11 00:34:53,063 : INFO : 已完成前 50000 行的斷詞
2019-05-11 00:34:53,409 : INFO : 已完成前 60000 行的斷詞
2019-0

In [ ]:
file_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\word2vec_model\new_merge_ptt.txt"
save_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\word2vec_model\model_wordvec"
ML_utils.Word2vec_train(file_path=file_path,save_path=save_path,model_min_count=2)

In [ ]:
save_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\word2vec_model\model_wordvec"
model = word2vec.Word2Vec.load(save_path)

In [ ]:
model.wv.most_similar(positive='難波萬',topn=20)

In [ ]:
model.wv.index2word